In [129]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from urllib.parse import quote
import datetime
# import seaborn as sns
from datetime import datetime, timedelta
import datetime
import matplotlib.pyplot as plt
# import itertools
# import statsmodels.api as sm
# from pylab import rcParams

import warnings; 
warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore",category=UserWarning)
warnings.simplefilter(action="ignore",category=FutureWarning)
warnings.filterwarnings("ignore")

# plt.rcParams['figure.figsize'] = (10, 10)
# plt.rcParams['grid.linestyle'] = ':'   
# plt.rcParams['axes.grid'] = True

# sns.set_style("whitegrid", {'axes.grid' : False})
#sns.color_palette("RdBu", n_colors=10)

np.float_ = np.float64

# %matplotlib inline
# %config InlineBackend.figure_formats = {'png', 'retina'}


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import math
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen


print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)
# print('Seaborn version', sns.__version__)

ImportError: DLL load failed while importing _c_internal_utils: The specified module could not be found.

In [38]:
cur_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("current times:", cur_time)

current times: 2024-08-04 22:35:52


In [119]:
dengue_x = pd.read_csv('../r506_pipeline/data/new_case.csv')
dengue_y = pd.read_csv('../temp_pipeline/data/dataset/temp_all.csv')

# Merge the features and labels data
dengue_df = pd.merge(dengue_x, dengue_y, on=['station', 'date'])

In [120]:
dengue_df.head()

,station,date,total_case,temp
0,9601,2019-01-07,12,27.454255
1,9601,2019-01-14,9,27.487042
2,9601,2019-01-21,9,27.443323
3,9601,2019-01-28,6,27.272813
4,9601,2019-02-04,7,27.605421


In [122]:
# Select only the data for 9601
dengue_df = dengue_df[dengue_df['station'] == 9601]

In [123]:
dengue_df.drop(['station'], axis=1, inplace=True)
dengue_df['date'] = pd.to_datetime(dengue_df['date'])
dengue_df.set_index('date', inplace=True)

In [124]:
dengue_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 260 entries, 2019-01-07 to 2023-12-25
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_case  260 non-null    int64  
 1   temp        260 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 6.1 KB


In [125]:
dengue_df = dengue_df.resample('W-MON').mean().interpolate(method='linear')

In [126]:
def get_forecast_accuracy(y, y_hat):
    # ME
    me = (y - y_hat).sum()/len(y)

    # RMSE
    rmse = math.sqrt(mean_squared_error(y, y_hat))

    # MAPE
    mape = mean_absolute_percentage_error(y, y_hat) * 100
    
    # WAPE
    wape = (y - y_hat).__abs__().sum() / y.__abs__().sum() * 100

    print("ME: %.2f, RMSE: %.2f, MAPE: %.2f%%, WAPE: %.2f%%" % (me,rmse,mape,wape))

In [127]:
# create timeseries readable by fbprophet
ts = pd.DataFrame({'ds':dengue_df.index,'y':dengue_df.total_case})

# Need to have bounds if using logistc growth. Unused for linear growth.
#ts['cap'] = 3000000
#ts['floor'] = 0 

ts.describe()

,ds,y
count,260,260.000000
mean,2021-07-01 12:00:00,4.726923
min,2019-01-07 00:00:00,0.000000
25%,2020-04-04 06:00:00,0.000000
50%,2021-07-01 12:00:00,2.000000
75%,2022-09-27 18:00:00,7.000000
max,2023-12-25 00:00:00,31.000000
std,NaN,5.953362


In [128]:
ts['y'].plot(figsize=(10,4))

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [ ]:
# instantiate the model and fit the timeseries
prophet = Prophet(yearly_seasonality=True, 
                  daily_seasonality=True, 
                  weekly_seasonality=True,
                  growth='linear',
                  n_changepoints= 10,
                  changepoint_range=1, # Consider the whole range for change points
                  changepoint_prior_scale=0.4) # Adjust trend flexibility if it overfits (too large) or underfits (too small).
                                               # Higher value = More flexible trend
prophet.fit(ts)

# Create additional points 25 periods into a future data frame 
future = prophet.make_future_dataframe(periods=25)
future['cap'] = 1
future['floor'] = 0
forecast = prophet.predict(future)

# plot
fig = prophet.plot(forecast)
ax = fig.gca()  # Now you have the matplotlib Axis
ax.legend(['Actual','Predicted','Bounds'])  
pass; 

In [93]:
# display the most critical output columns from the forecast
forecast[['ds','yhat','yhat_lower','yhat_upper']].head()

,ds,yhat,yhat_lower,yhat_upper
0,2019-01-07,1.230510,0.408754,2.030151
1,2019-01-14,1.401555,0.566249,2.239511
2,2019-01-21,1.421406,0.566362,2.246195
3,2019-01-28,1.433436,0.551693,2.278736
4,2019-02-04,1.541755,0.635141,2.391468


In [114]:
dengue_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   index       261 non-null    int64         
 1   date        261 non-null    datetime64[ns]
 2   total_case  261 non-null    float64       
 3   temp        261 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 8.3 KB


In [117]:
dengue_df.head()

,date,total_case,temp
0,2019-01-07,2.600000,27.618841
1,2019-01-14,2.166667,27.559720
2,2019-01-21,1.416667,27.510088
3,2019-01-28,1.461538,27.272813
4,2019-02-04,1.615385,27.605421


In [115]:
# dengue_df.drop(['index'], axis=1, inplace=True)

In [ ]:
# dengue_df.to_csv(r"data\case_allv2.csv")